In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import pywikibot as pb
from pywikibot.specialbots import UploadRobot

import requests
from requests.compat import quote
from bs4 import BeautifulSoup
from mako.template import Template

import os, re
import shutil
import calendar

commons_site = pb.Site("commons", "commons")

In [2]:
# Path handling for importing utils.py
import sys, inspect
current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
folder_parts = current_folder.split(os.sep)
parent_folder = os.sep.join(folder_parts[:-1])

if current_folder not in sys.path:
    sys.path.insert(0, current_folder)
if parent_folder not in sys.path:
    sys.path.insert(0, parent_folder)
    
from wikimedia.utils import is_commons_file, get_hash

In [3]:
# Creation of images folder
cwd = os.getcwd()

images_directory = os.path.join(cwd, 'images')
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

In [36]:
#### User input:
url = 'https://diario.madrid.es/blog/notas-de-prensa/el-ayuntamiento-homenajea-con-una-calle-a-soledad-cazorla-impulsora-de-la-ley-contra-la-violencia-de-genero/'

user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1'
headers = {'User-Agent' : user_agent}

#### User input:
categories = ['Calle de Soledad Cazorla, Madrid',
              'Francisca Sauquillo',
              'Celia Mayer',
              'Purificación Causapié',
              'Carlos Mato',
              'Mar Espinar']

In [37]:
#### User input:
upload_categories = ['Files uploaded by User:Discasto']
categories = categories + upload_categories
categories

['Calle de Soledad Cazorla, Madrid',
 'Francisca Sauquillo',
 'Celia Mayer',
 'Purificación Causapié',
 'Carlos Mato',
 'Mar Espinar',
 'Files uploaded by User:Discasto']

In [38]:
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')

In [39]:
title = soup.find_all("h1", class_="post-title")[0].get_text().strip().replace('  ', ' ')
title

'El Ayuntamiento homenajea con una calle a Soledad Cazorla, impulsora de la ley contra la violencia de género'

In [40]:
date = '-'.join(soup.find_all("div", class_="post-date")[0].get_text().strip().split('/')[::-1])
#date = ""
year = date.split('-')[0]
month = calendar.month_name[int(date.split('-')[1])]
date

'2018-06-10'

In [41]:
body = soup.find_all("div", class_="post-content")
p_description = body[0].find_all("p")
for p in p_description :
    if len(p.get_text()) > 10:
        description = p.get_text()
        break
#description=u'Una delegación del Ayuntamiento de Madrid asiste a la manifestación de Barcelona en contra de los atentados terroristas, presidida por Manuela Carmena, y con representación de todos los grupos políticos municipales. Esta representación municipal ha estado compuesta por la portavoz del gobierno municipal, Rita Maestre: el delegado de Coordinación Territorial y Cooperación Público-Social y concejal del Grupo Municipal Ahora Madrid, Nacho Murgui; la concejala y delegada del portavoz del Grupo Municipal Partido Popular, Isabel Rosell; el concejal y delegado de la portavoz del Grupo Municipal Socialista, Ignacio de Benito y la portavoz del Grupo Municipal Ciudadanos-Partido de la Ciudadanía, Begoña Villacís.\n"Madrid como ciudad está aquí, con Barcelona, por solidaridad y por la necesidad de expresar la protesta que significan los actos de esta barbarie, para hacer posible que no se vuelvan a repetir", ha dicho la alcaldesa a su llegada a la capital catalana.'
description

'El Ayuntamiento de Madrid ha rendido homenaje este domingo a Soledad Cazorla, la primera mujer fiscal de sala contra la violencia sobre la mujer, en la calle bautizada recientemente con su nombre. Con rosas blancas, las flores preferidas de la jurista, y un poema ha concluido un acto al que acudieron niños y niñas becadas a través del Fondo Soledad Cazorla para menores víctimas de la violencia machista.'

In [42]:
template = u"""=={{int:filedesc}}==
{{Information
|description={{es|1=${description}}}
|date=${date}
|source=[${url} Diario de Madrid - ${title}]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=[https://diario.madrid.es/contenidos-libres/ License information for all contents in diario.madrid.es]
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Images from Ayuntamiento de Madrid (to classify)]]
[[Category:${month} ${year} in Madrid]]
${cat_string}"""

vars = {
    "url": url,
    "description": description,
    "year": year,
    "month": month,
    "date": date,
    "title": title,
    "cat_string": '\n'.join(['[[Category:'+i+']]' for i in categories])
}
t = Template(template)
_text = t.render(**vars)
_text

'=={{int:filedesc}}==\n{{Information\n|description={{es|1=El Ayuntamiento de Madrid ha rendido homenaje este domingo a Soledad Cazorla, la primera mujer fiscal de sala contra la violencia sobre la mujer, en la calle bautizada recientemente con su nombre. Con rosas blancas, las flores preferidas de la jurista, y un poema ha concluido un acto al que acudieron niños y niñas becadas a través del Fondo Soledad Cazorla para menores víctimas de la violencia machista.}}\n|date=2018-06-10\n|source=[https://diario.madrid.es/blog/notas-de-prensa/el-ayuntamiento-homenajea-con-una-calle-a-soledad-cazorla-impulsora-de-la-ley-contra-la-violencia-de-genero/ Diario de Madrid - El Ayuntamiento homenajea con una calle a Soledad Cazorla, impulsora de la ley contra la violencia de género]\n|author=[https://diario.madrid.es/ Diario de Madrid]\n|permission=[https://diario.madrid.es/contenidos-libres/ License information for all contents in diario.madrid.es]\n|other versions=\n}}\n\n=={{int:license-header}}==

In [43]:
images = [image.a["href"] for image in soup.find_all("div", class_="gallery-icon")]
images

['https://diario.madrid.es/wp-content/uploads/2018/06/cazorla1-1189x1000.jpg',
 'https://diario.madrid.es/wp-content/uploads/2018/06/cazorla2-1500x990.jpg',
 'https://diario.madrid.es/wp-content/uploads/2018/06/cazorla4-1424x1000.jpg',
 'https://diario.madrid.es/wp-content/uploads/2018/06/cazorla5-1333x1000.jpg',
 'https://diario.madrid.es/wp-content/uploads/2018/06/Cazorla6.jpg']

In [44]:
#### User input:
excluded = []

used_names = []
counter = 0

for i, image in enumerate(images):
    # If the image is excluded, skip
    if i in excluded:
        print ("Image excluded. Skipping")
        continue
        
    # First, the image is downloaded and stored
    image_url = quote(image.encode('utf-8'), ':/')
    image_name = '{} {:02d}.jpg'.format(title.replace(':', ' -').replace('  ', ' '), counter+1)
    image_path = os.path.join(images_directory, image_name)
    try: 
        r = requests.get(image_url, headers=headers, stream=True)
        with open(image_path, 'wb') as out_file:
            shutil.copyfileobj(r.raw, out_file)
    except :
        print ('Failed download. Skipping')
        continue

    # If the image is already in Commons, skip
    if is_commons_file(get_hash(image_path)) :
        print ("Image already in commons. Skipping")
        os.remove(image_path)
        continue

    # If the image name is already in commons, find a new name
    if pb.Page(commons_site, image_name, ns=6).exists():
        print ("Image name ({0}) already used in Commons".format(image_name))
        used_names.append(image_name)
        
    while True:
        if image_name in used_names :
            # Finding a new name
            image_subject = '.'.join(image_name.split('.')[:-1])
            image_extension = 'jpg'
            p = re.compile('(.*) ([0-9]{2}\.jpg)')
            m = p.match(image_name)
            if m is None:
                image_name = image_subject + ' 01.' + image_extension
            else :
                counter = int(m.group(2)[:2]) + 1
                image_name = '{} {:02d}.{}'.format(m.group(1), counter, image_extension)

            if pb.Page(commons_site, image_name, ns=6).exists():
                print ("Image name ({0}) already used in Commons. Finding a new name".format(image_name))
                used_names.append(image_name)
        else :
            print ("Preparing to upload image with name {0}".format(image_name))
            used_names.append(image_name)
            break

    # image upload
    bot = UploadRobot([image_path],
                      description = _text,
                      useFilename = image_name,
                      keepFilename = True,
                      verifyDescription = False,
                      ignoreWarning = True,
                      targetSite = commons_site)
    bot.run()

    os.remove(image_path)

Preparing to upload image with name El Ayuntamiento homenajea con una calle a Soledad Cazorla, impulsora de la ley contra la violencia de género 01.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=El Ayuntamiento de Madrid ha rendido homenaje este domingo a Soledad Cazorla, la primera mujer fiscal de sala contra la violencia sobre la mujer, en la calle bautizada recientemente con su nombre. Con rosas blancas, las flores preferidas de la jurista, y un poema ha concluido un acto al que acudieron niños y niñas becadas a través del Fondo Soledad Cazorla para menores víctimas de la violencia machista.}}
|date=2018-06-10
|source=[https://diario.madrid.es/blog/notas-de-prensa/el-ayuntamiento-homenajea-con-una-calle-a-soledad-cazorla-impulsora-de-la-ley-contra-la-violencia-de-genero/ Diario de Madrid - El Ayuntamiento homenajea con una calle a Soledad Cazorla, impulsora de la ley contra la violencia de género]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=[https://diario.madrid.es/contenidos-libres/ License information for all contents in diario.madrid.es]
|other versions=
}}

=={{int

Image name (El Ayuntamiento homenajea con una calle a Soledad Cazorla, impulsora de la ley contra la violencia de género 01.jpg) already used in Commons
Preparing to upload image with name El Ayuntamiento homenajea con una calle a Soledad Cazorla, impulsora de la ley contra la violencia de género 02.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=El Ayuntamiento de Madrid ha rendido homenaje este domingo a Soledad Cazorla, la primera mujer fiscal de sala contra la violencia sobre la mujer, en la calle bautizada recientemente con su nombre. Con rosas blancas, las flores preferidas de la jurista, y un poema ha concluido un acto al que acudieron niños y niñas becadas a través del Fondo Soledad Cazorla para menores víctimas de la violencia machista.}}
|date=2018-06-10
|source=[https://diario.madrid.es/blog/notas-de-prensa/el-ayuntamiento-homenajea-con-una-calle-a-soledad-cazorla-impulsora-de-la-ley-contra-la-violencia-de-genero/ Diario de Madrid - El Ayuntamiento homenajea con una calle a Soledad Cazorla, impulsora de la ley contra la violencia de género]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=[https://diario.madrid.es/contenidos-libres/ License information for all contents in diario.madrid.es]
|other versions=
}}

=={{int

Preparing to upload image with name El Ayuntamiento homenajea con una calle a Soledad Cazorla, impulsora de la ley contra la violencia de género 03.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=El Ayuntamiento de Madrid ha rendido homenaje este domingo a Soledad Cazorla, la primera mujer fiscal de sala contra la violencia sobre la mujer, en la calle bautizada recientemente con su nombre. Con rosas blancas, las flores preferidas de la jurista, y un poema ha concluido un acto al que acudieron niños y niñas becadas a través del Fondo Soledad Cazorla para menores víctimas de la violencia machista.}}
|date=2018-06-10
|source=[https://diario.madrid.es/blog/notas-de-prensa/el-ayuntamiento-homenajea-con-una-calle-a-soledad-cazorla-impulsora-de-la-ley-contra-la-violencia-de-genero/ Diario de Madrid - El Ayuntamiento homenajea con una calle a Soledad Cazorla, impulsora de la ley contra la violencia de género]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=[https://diario.madrid.es/contenidos-libres/ License information for all contents in diario.madrid.es]
|other versions=
}}

=={{int

Image name (El Ayuntamiento homenajea con una calle a Soledad Cazorla, impulsora de la ley contra la violencia de género 03.jpg) already used in Commons
Preparing to upload image with name El Ayuntamiento homenajea con una calle a Soledad Cazorla, impulsora de la ley contra la violencia de género 04.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=El Ayuntamiento de Madrid ha rendido homenaje este domingo a Soledad Cazorla, la primera mujer fiscal de sala contra la violencia sobre la mujer, en la calle bautizada recientemente con su nombre. Con rosas blancas, las flores preferidas de la jurista, y un poema ha concluido un acto al que acudieron niños y niñas becadas a través del Fondo Soledad Cazorla para menores víctimas de la violencia machista.}}
|date=2018-06-10
|source=[https://diario.madrid.es/blog/notas-de-prensa/el-ayuntamiento-homenajea-con-una-calle-a-soledad-cazorla-impulsora-de-la-ley-contra-la-violencia-de-genero/ Diario de Madrid - El Ayuntamiento homenajea con una calle a Soledad Cazorla, impulsora de la ley contra la violencia de género]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=[https://diario.madrid.es/contenidos-libres/ License information for all contents in diario.madrid.es]
|other versions=
}}

=={{int

Preparing to upload image with name El Ayuntamiento homenajea con una calle a Soledad Cazorla, impulsora de la ley contra la violencia de género 05.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=El Ayuntamiento de Madrid ha rendido homenaje este domingo a Soledad Cazorla, la primera mujer fiscal de sala contra la violencia sobre la mujer, en la calle bautizada recientemente con su nombre. Con rosas blancas, las flores preferidas de la jurista, y un poema ha concluido un acto al que acudieron niños y niñas becadas a través del Fondo Soledad Cazorla para menores víctimas de la violencia machista.}}
|date=2018-06-10
|source=[https://diario.madrid.es/blog/notas-de-prensa/el-ayuntamiento-homenajea-con-una-calle-a-soledad-cazorla-impulsora-de-la-ley-contra-la-violencia-de-genero/ Diario de Madrid - El Ayuntamiento homenajea con una calle a Soledad Cazorla, impulsora de la ley contra la violencia de género]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=[https://diario.madrid.es/contenidos-libres/ License information for all contents in diario.madrid.es]
|other versions=
}}

=={{int